In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm.auto import tqdm
import proplot as plot
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr, maria_IO
from tools.preprocess import do_eof,preproc_maria,preproc_haiyan
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

/tmp/ipykernel_430177/1773301035.py:20: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


In [2]:
def Pipeline(case='Maria',TYPE='3D' or '2D',suffix=None):
    
    def produce_Xy(PCAdict=None,maria_data=None,nummem=None,divider=None,EXP=None,storename=None):
        # Normalized time series (X)
        maria_pca_nonorml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).produce_timeseries( maria_data )
        maria_pca_norml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).normalize_timeseries( maria_pca_nonorml )
        # Back to experiments for splitting
        exp_pca_norml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).back_to_exp( maria_pca_norml, divider ) 
        exp_pca_nonorml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).back_to_exp( maria_pca_nonorml, divider ) 
        
        X_dict = {}
        for ind,obj in enumerate(storename):
            test1 = (maria_IO.input_output(dict1,None,['u','v','w','qv','theta','heatsum','hdia','rad','ir'],None).make_X_nosep( exp_pca_norml, EXP[ind]))
            X_dict[obj] = test1
        yall = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).make_Y_nosep( exp_pca_nonorml, np.linspace(0,35,36)+1)  
        return X_dict,yall
    
    if case=='Maria':
        filepath = input('Enter Path:')
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Import Preprocessed Flat Data and PCA dictionary
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            tempdict = {}
            for varname in ['u','v','w','qv','theta','heatsum','hdia','rad','ir']:
                temp = [read_and_proc.depickle(filepath+str(lime)+suffix)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict[varname] = maria_IO.long_MariaExps(temp)[0]
        elif TYPE=='2D':
            tempdict = {}
            for varname in ['u','v','w','qv','theta','heatsum','hdia','rad','ir']:
                temp = [read_and_proc.depickle(filepath+str(lime)+suffix)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict[varname] = maria_IO.long_MariaExps(maria_IO.to_azim(temp))[0]            
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Save to xarray
        #------------------------------------------------------------------------------------------------------------------------------------------
        dims = ['sample','flatarray']
        coords = dict(sample=np.linspace(0,tempdict['u'].shape[0]-1,tempdict['u'].shape[0]),flatarray=np.linspace(0,tempdict['u'].shape[1]-1,tempdict['u'].shape[1]))
        ds = xr.Dataset(coords=coords)
        maria_data=preproc_haiyan.build_a_xarray_dataset(ds=ds,varname=['u','v','w','theta','heatsum','hdia','rad','qv','ir'],\
                                                         varfile=[tempdict['u'],tempdict['v'],tempdict['w'],tempdict['theta'],tempdict['heatsum'],tempdict['hdia'],tempdict['rad'],tempdict['qv'],tempdict['ir']],dims=dims,coords=coords)
        del tempdict
        gc.collect()
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Import PCA dictionary
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/processed/'
            dict1 = read_and_proc.depickle(folderpath+'PCA/PCAdict2')
        elif TYPE=='2D':
            folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/processed/intermediate/'
            dict1 = read_and_proc.depickle(folderpath+'PCA/PCAdict')
            
        print("--Start producing Input / Output--")
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Create Input/ output
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            nummem=[54,26,50,5,75,12,12,10,10]
        elif TYPE=='2D':
            nummem = [20,11,20,5,12,7,7,5,5]
        EXP = [['hdia','rad'],['theta','hdia','rad'],['u','v','w','theta','hdia','rad'],['u','v','theta','hdia','rad'],['u','v'],['u','v','w'],['u','v','hdia','rad'],['u','v','w','hdia','rad'],['u','v','w','theta','qv','hdia','rad']]
        storename = ['dt','dtth','dtthuvw','dtthuv','uv','uvw','dtuv','dtuvw','dtthuvwqv']
        X_dict,yall = produce_Xy(dict1, maria_data, nummem, divider, EXP, storename)
        
        return maria_data, dict1, X_dict, yall

In [3]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'
enter = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'

maria_u = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['u'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
divider = np.asarray([maria_u[0][12:].shape[0],maria_u[1][36:].shape[0],maria_u[2][60:].shape[0],maria_u[3][96:].shape[0],maria_u[4][36:].shape[0]]).cumsum()

  0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
maria_data, dict1, X, y = Pipeline('Maria','2D','_smooth_preproc_dict1b_g')

Enter Path: /work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

--Start producing Input / Output--


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [6]:
suffix='_smooth_preproc_dict1b_g_dudvdwdth'
dudvdwdth = {}
for ind,obj in enumerate(['du','dv','dw','dtheta']):
    array = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)[obj] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
    tempn = [np.nanmean(array[0].reshape(144,39,360,200),axis=2).reshape(144,39*200),np.nanmean(array[1].reshape(144,39,360,200),axis=2).reshape(144,39*200),\
             np.nanmean(array[2].reshape(144,39,360,200),axis=2).reshape(144,39*200),np.nanmean(array[3].reshape(144,39,360,200),axis=2).reshape(144,39*200),\
            np.nanmean(array[4].reshape(144,39,360,200),axis=2).reshape(144,39*200)]
    temp = [tempn[0][12:,:],tempn[1][36:,:],tempn[2][60:,:],tempn[3][96:,:],tempn[4][36:,:]]
    dudvdwdth[obj] = np.concatenate(temp,axis=0)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
#datacheck(b,a,divider).dudvdwVAR(dudvdwdth,'w')
#[np.abs(b[obj].explained_variance_ratio_.cumsum()-0.91).argmin() for obj in ['u','v','w','qv','theta','heatsum','hdia','rad','ir']]

[7, 0, 7, 4, 2, 6, 6, 4, 4]

In [ ]:
aaaa = xr.open_dataset('/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/maria/U.nc')
fig,axs = plot.subplots(ncols=3,nrows=1,axwidth=4.295,refaspect=1.431)
for inx,i in enumerate([0,1,2]):
    pcw=axs[inx].contourf(np.linspace(0,199,200)*3,aaaa.pres.data,
                   b['rad'].components_[i,:].reshape(39,200),cmap='balance',levels=np.linspace(-0.03,0.03,21),extend='both')
    axs[inx].colorbar(pcw,loc='ur',ticks=0.01)
    axs[inx].format(title=f'RAD PC#{str(i+1)}, Azimuthal Average')
axs.format(xlabel=r'X (km)',ylabel='Y (km)',suptitle='Maria')
#plt.savefig('../../figures/0727/maria_important_RAD_azimavg.png',dpi=300)
plt.show()

#### Create input/output

In [16]:
nummem = [20,11,20,5,12,7,7,5,5]
X_dict_sep = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem )._back_to_exp( X['dtthuvwqv'], divider ) 
yall_sep = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem )._back_to_exp( y[5], divider ) 

In [17]:
def create_index(seed=None,explength=[133,109,85,49,109],percentage=None,findnum=49):
    rng = np.random.default_rng(seed)
    return [rng.choice(np.linspace(0,explength[i]-1,explength[i]),int(np.round(findnum*percentage[i])),replace=False) for i in range(5)]

def sep_train_valid(varlist=None,explength=[133,109,85,49,109]):
    def _out_index(length=None,index=None):
        return [int(obj) for obj in index], [int(obj) for obj in np.linspace(0,int(length-1),int(length)) if obj not in index]
    trainstore,validstore = [],[]
    for i in range(len(varlist)):
        validindex,trainindex = _out_index(explength[i],indices[i])
        validdata,traindata = varlist[i][validindex],varlist[i][trainindex]
        trainstore.append(traindata)
        validstore.append(validdata)
    return trainstore,validstore

In [27]:
# 3 done
Xtrain = np.concatenate([X_dict_sep[i] for i in [0,2,3,4]],axis=0)
Xvalid = (X_dict_sep[1])
ytrain = (np.concatenate([yall_sep[i] for i in [0,2,3,4]],axis=0))
yvalid = (yall_sep[1])

In [28]:
def delete_padding(inTS=None,outTS=None):
    output_nozero,input_nozero = [],[]
    for i in range(len(outTS[:,0])):
        temp = outTS[i,:]
        tempin = inTS[i,:]
        if temp.all()==0:
            continue
        else:
            output_nozero.append(temp)
            input_nozero.append(tempin)
    return input_nozero,output_nozero

In [29]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/processed/intermediate/'
read_and_proc.save_to_pickle(folderpath+'inputoutput/Xtrain_6_test1',Xtrain,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput/Xvalid_6_test1',Xvalid,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput/ytrain_6_test1',ytrain,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput/yvalid_6_test1',yvalid,'PICKLE')

In [98]:
np.asarray(delete_padding(Xtrain[0],ytrain[0])[0]).shape

(322, 232)

In [29]:
np.asarray(delete_padding(Xtrain,ytrain)[0]).shape

(340, 232)